<a href="https://colab.research.google.com/github/Alvaro-Tocchio/monitorias/blob/master/Prova_final_Bianca_Alvaro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Prova Final**

Alvaro Tocchio nºUSP 8964000 <br>
Bianca de Oliveira Araujo nºUSP 8964852


In [ ]:
#imports
import numpy as np
import math
from scipy import optimize as opt
from scipy.optimize import linprog

#### **1.1 Exercício 1**
##### **1.1.1 1a** 

In [ ]:
def partition(m,v):
  lower = list()
  upper = list()
  equal = list()

  for e in m:
    if e > v:
      upper.append(e)
    elif e < v:
      lower.append(e)
    else:
      equal.append(e)
  return lower, equal, upper

##### **1.1.1 1b**

In [ ]:
def qsort(m):
  if len(m)>1:
    value = m[0]
    lower, equal, upper = partition(m, value)
    #recursão ordena a parte menor e a maior
    l = qsort(lower)
    u = qsort(upper)
    #junta as listas ordenadas
    return l + equal + u
  else:
    #retorna a lista se ela tiver 1 ou nenhum elemento
    return m 



#### **1.2 Exercício 2**



In [ ]:
def deriv(f, x, i, h=1e-6):
  x = np.array(x)
  x_menos = np.array(x)
  x_mais = np.array(x)

  #varia o indice 'i' em h para calcular a diferença
  x_menos[i] -= h
  x_mais[i] += h

  e = (f(*x_mais) - f(*x_menos)) / (2*h)
  return e


#### **1.3 Exercício 3**

In [ ]:
def definiteness(A):
  autovalores = np.linalg.eigvals(A)
  if np.all(autovalores > 0):
    return 2
  elif np.all(autovalores >= 0):
    return 1
  elif np.all(autovalores < 0):
    return -2
  elif np.all(autovalores <= 0):
    return -1
  else:
    return 0

#### **1.4 Exercício 4**

In [ ]:
# funcao original
# -(x**6) + 3*(x**3) + 4*x -1

def f(x):
  # funcao -f para possuir um minimo
  return (x**6) - 3(x**3) - 4*x + 1

def optf():
  # basinhopping encontra o minimo de -f
  # -1 * minimo de -f é o maximo de f

  x_maximo = opt.basinhopping(f,0).x
  return -f(x_maximo)
  

#### **1.5 Exercício 5**

In [ ]:
# Ax -> Número de barris indo de texas para chicago
# Ay -> Número de barris indo da california para chicago
# Az -> Número de barris indo do alasca para chicago
# Bx -> Número de barris indo de texas para inglaterra
# By -> Número de barris indo da california para inglaterra
# Az -> Número de barris indo do alasca para inglaterra

# Funcao a ser minimizada: 1000Ax + 2000Ay + 3000Az + 1500Bx + 3000By + 3700Bz

# Restricoes:
# Ax + Bx <= 1000000 |
# Ay + By <= 1000000 | -> Cada fonte pode mandar até 1000000
# Az + Bz <= 1000000 |
# Ax + Ay + Az = 800000 -> 800000 indo para chicago
# Bx + By + Bz = 2200000 -> 2200000 indo para nova inglaterra

def oil():
  c = [1000, 2000, 3000, 1500, 3000, 3700]
  
  A_ub = [[1,0,0,1,0,0],
          [0,1,0,0,1,0],
          [0,0,1,0,0,1]]
  b_ub = [1000000, 1000000, 1000000]


  A_eq = [[1,1,1,0,0,0],
          [0,0,0,1,1,1]]
  b_eq = [800000, 2200000]

  res = linprog(c, A_ub = A_ub, b_ub = b_ub, A_eq = A_eq, b_eq = b_eq,
                method = 'revised simplex')
  valores = res.x

  sol = {
      'CH': {'TX': valores[0], 'CA': valores[1], 'AK': valores[2]}
      'NE': {'TX': valores[3], 'CA': valores[4], 'AK': valores[5]}
      }

  return sol


          




#### **1.6 Exercício 6**

In [ ]:
def J(k):
    grafo = {
        'A': {'C': 5, 'B': 1, 'D': 3},
        'B': {'D': 9, 'E': 6},
        'C': {'F': 2},
        'D': {'F': 4, 'G': 8},
        'E': {'G': 4},
        'F': {'G': 1},
        'G': {}
    }
    # grafo com menor distancia
    j = {}
    nao_visitados = grafo

    # inicializa a menor ditancia como infinito
    # para todos os nós
    for vertice in  nao_visitados:
        j[vertice] = math.inf

    j[k] = 0
    while(nao_visitados):
        melhor_vertice = None
        
        for vertice in nao_visitados: 
            if melhor_vertice is None:
                melhor_vertice = vertice
                
            elif j[melhor_vertice] > j[vertice]:
                melhor_vertice = vertice

        for vizinho, custo in nao_visitados[melhor_vertice].items():
            if custo + j[melhor_vertice] < j[vizinho]:  
                j[vizinho] = custo + j[melhor_vertice]

        nao_visitados.pop(melhor_vertice)
    # retorna o menor custo até G
    return j['G']


#### **1.7 Exercício 7**

In [ ]:
# função produção de cerveja
def Y(x, alpha):
    return x**alpha


# retorna o preço no instante t
def get_pt(p, t, gamma):
    if t == 1:
        return 1 + gamma
    if p[t] != 0:
        return p[t]
    return (1 + gamma) * get_pt(p, t-1, gamma)


# derivada do lucro
def df(x, t, alpha, gamma, r, p):
    p[t] = get_pt(p, t, gamma)
    return (alpha * ((1/(1+r)) * p[t] * Y(x, alpha-1)))


# Função lucro
def f(x, t, alpha, gamma, r, p):
    p[t] = get_pt(p, t, gamma)
    return ((1/(1+r)) * p[t] * Y(x, alpha))


def beer(X0, alpha, gamma, r):
    p = np.zeros(1000)
    p[0] = 1
    # encontra a raíz da derivada
    res = opt.root(df, [0], args=(1000, alpha, gamma, r, p))
    # retorna a produção máxima
    return Y(res.x, alpha)

#### **1.8 Exercício 8**

In [ ]:
def V(u, p, Y):
    # nega U para minimizar com restrição
    def menos_u(x):
        return -1 * u(x)

    plinha = np.diff(p)
    # encontra minimo de -U(x) da função
    res = opt.minimize(menos_u, [0])
    # retorna o valor da utilidade encontrada
    return -res.fun